In [2]:
import graphlab

In [3]:
train_data = graphlab.SFrame('image_train_data/')
test_data = graphlab.SFrame('image_test_data/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1538730685.log


This non-commercial license of GraphLab Create for academic use is assigned to r38411@yahoo.com and will expire on August 31, 2019.


In [4]:
train_data.print_rows(num_rows=2)

+----+----------------------+-------+-------------------------------+
| id |        image         | label |         deep_features         |
+----+----------------------+-------+-------------------------------+
| 24 | Height: 32 Width: 32 |  bird | [0.242871761322, 1.0954537... |
| 33 | Height: 32 Width: 32 |  cat  | [0.525087952614, 0.0, 0.0,... |
+----+----------------------+-------+-------------------------------+
+-------------------------------+
|          image_array          |
+-------------------------------+
| [73.0, 77.0, 58.0, 71.0, 6... |
| [7.0, 5.0, 8.0, 7.0, 5.0, ... |
+-------------------------------+
[2005 rows x 5 columns]



In [6]:
train_data['label'].sketch_summary()


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------+-----+-----+------+
| value | automobile | cat | dog | bird |
+-------+------------+-----+-----+------+
| count |    509     | 509 | 509 | 478  |
+-------+------------+-----+-----+------+


# Question 1:
# in label, there are 4 unique labels:
# automobile: 509, cat:509, dog:509, bird:478, total=2005

In [11]:
image_label = train_data['label'].unique()
image_label

dtype: str
Rows: 4
['bird', 'dog', 'cat', 'automobile']

In [12]:
## split the data into 4 categories

In [13]:
image_group = {}
for i in image_label:
    key_name = i + '_model'
    image_group[key_name] = train_data.filter_by(i, 'label')

In [14]:
image_group

{'automobile_model': Columns:
 	id	int
 	image	Image
 	label	str
 	deep_features	array
 	image_array	array
 
 Rows: 509
 
 Data:
 +------+----------------------+------------+-------------------------------+
 |  id  |        image         |   label    |         deep_features         |
 +------+----------------------+------------+-------------------------------+
 |  97  | Height: 32 Width: 32 | automobile | [1.57818555832, 0.0, 0.0, ... |
 | 136  | Height: 32 Width: 32 | automobile | [0.0, 0.0, 0.0, 0.0, 0.0, ... |
 | 302  | Height: 32 Width: 32 | automobile | [0.583938002586, 0.0, 0.0,... |
 | 312  | Height: 32 Width: 32 | automobile | [0.0, 0.0, 0.0, 0.39282339... |
 | 323  | Height: 32 Width: 32 | automobile | [0.0, 0.0, 0.0, 4.42310428... |
 | 536  | Height: 32 Width: 32 | automobile | [0.0, 0.0, 0.0, 0.0, 0.0, ... |
 | 593  | Height: 32 Width: 32 | automobile | [1.65033948421, 0.0, 0.0, ... |
 | 962  | Height: 32 Width: 32 | automobile | [0.0, 0.0, 0.0, 0.39552795... |
 | 997  | Hei

In [15]:
image_group.keys()

['bird_model', 'cat_model', 'dog_model', 'automobile_model']